# Búsquedas aproximadas con LSH (Locality Sensitive Hashing)

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025  

Instalar NearPy con:

```
  conda install nearpy
```

La documentación de NearPy está disponible en https://github.com/pixelogik/NearPy

Si se está usando anaconda, instalar con  `conda install -c conda-forge nearpy`

In [ ]:
# primero cargar algunas funciones auxiliares
import numpy
import time
import pyflann


# calcular la calidad de la respuesta contra el resultado "ideal" dado por el linear scan
class Experimento:
    def __init__(self, nombre, archivo_dataset_q, archivo_dataset_r):
        self.nombre = nombre
        # cargar conjuntos de vectores Q y R
        self.dataset_q = numpy.load(archivo_dataset_q)
        self.dataset_r = numpy.load(archivo_dataset_r)
        print(
            "{}: conjunto_Q={} conjunto_R={}".format(
                nombre, self.dataset_q.shape, self.dataset_r.shape
            )
        )

    def calcular_linear_scan(self):
        # crear un objeto flann
        flann = pyflann.FLANN()
        # construir el indice para linear scan
        flann.build_index(self.dataset_r, algorithm="linear")
        # ejecutar el linear scan, se usa un solo core, debiera demorar unos 25 a 35 segundos
        t0 = time.time()
        nns, dists = flann.nn_index(experimentoA.dataset_q, num_neighbors=1, cores=1)
        # se guarda el objeto encontrado, no se guarda la distancia porque LSH no la calcula
        self.gt_nns = nns
        self.gt_tiempo = time.time() - t0
        self.correctas = 0
        self.incorrectas = 0
        self.tiempo = 0
        self.precision = 0
        self.eficiencia = 0
        print(
            "{}: Linear Scan toma {:.1f} segundos".format(self.nombre, self.gt_tiempo)
        )

    def evaluar_busqueda(self, nombre_indice, nns, segundos):
        self.correctas = 0
        self.incorrectas = 0
        for i in range(len(self.gt_nns)):
            # es correcta si encontró el mismo elemento que el linear scan
            if nns[i] == self.gt_nns[i]:
                self.correctas += 1
            else:
                self.incorrectas += 1
        self.tiempo = segundos
        self.precision = self.correctas / (self.correctas + self.incorrectas)
        self.eficiencia = self.tiempo / self.gt_tiempo
        print(
            "{:<38}: correctas={:>4} incorrectas={:>4} precision={:>6.1%} tiempo={:>4.1f} seg. ({:>5.1%} de LS)".format(
                nombre_indice,
                self.correctas,
                self.incorrectas,
                self.precision,
                self.tiempo,
                self.eficiencia,
            )
        )


experimentoA = Experimento("DATASET_A", "dataset_a_q.npy", "dataset_a_r.npy")
experimentoA.calcular_linear_scan()

# Construir índice LSH y resolver búsquedas kNN

In [ ]:
import nearpy


def crear_indice_lsh(dataset_r, cantidad_hashes, projection_count, bin_width):
    t0 = time.time()
    # definir las funciones de hash
    lsh_funciones = []
    for i in range(cantidad_hashes):
        num = len(lsh_funciones)
        nombre = "hash_" + str(num)
        # crear la funcion de hash
        func_hash = nearpy.hashes.RandomDiscretizedProjections(
            hash_name=nombre,
            projection_count=projection_count,
            bin_width=bin_width,
            rand_seed=num,
        )
        # agregarla a la lista
        lsh_funciones.append(func_hash)
    # crear estructura
    lsh_engine = nearpy.Engine(
        dim=dataset_r.shape[1],
        lshashes=lsh_funciones,
        distance=nearpy.distances.EuclideanDistance(),
        vector_filters=[nearpy.filters.nearestfilter.NearestFilter(1)],
    )
    # construir el indice
    for i in range(dataset_r.shape[0]):
        lsh_engine.store_vector(dataset_r[i], i)
    segundos = time.time() - t0
    return lsh_engine, segundos


def busqueda_NN_lsh(dataset_q, lsh_engine):
    t0 = time.time()
    nn_results = []
    for i in range(dataset_q.shape[0]):
        results = lsh_engine.neighbours(dataset_q[i])
        id_vector_nn = -1
        if len(results) > 0:
            nn_data = results[0]
            id_vector_nn = nn_data[1]
        nn_results.append(id_vector_nn)
    segundos = time.time() - t0
    return nn_results, segundos


def indexar_y_evaluar_lsh(experimento, cantidad_hashes, projection_count, bin_width):
    nombre_indice = (
        f"LSH hashes={cantidad_hashes} proyecciones={projection_count} bins={bin_width}"
    )
    print("{}: construyendo indice {}".format(experimento.nombre, nombre_indice))
    lsh_engine, segundos = crear_indice_lsh(
        experimento.dataset_r, cantidad_hashes, projection_count, bin_width
    )
    print("    tiempo construccion: {:.1f} segundos".format(segundos))
    nns, segundos = busqueda_NN_lsh(experimento.dataset_q, lsh_engine)
    experimento.evaluar_busqueda(nombre_indice, nns, segundos)

### Probar una función LSH simple en ambos datasets

In [ ]:
cantidad_hashes = 3
projection_count = 10
bin_width = 10

indexar_y_evaluar_lsh(experimentoA, cantidad_hashes, projection_count, bin_width)

### Probando con distintos parámetros

In [ ]:
cont = 0
for cantidad_hashes in [5, 10]:
    for projection_count in [10, 15]:
        for bin_width in [300, 400, 500, 600]:
            cont += 1
            print()
            print("caso " + str(cont))
            indexar_y_evaluar_lsh(
                experimentoA, cantidad_hashes, projection_count, bin_width
            )

Las mejores configuraciones (obtenidas en mi computador, puede variar) son:
```
LSH hashes=5  proyecciones=10 bins=600: correctas=3415 incorrectas=1585 precision=68.3%	tiempo=8.41	seg. (27.0% del tiempo de Linear Scan)
LSH hashes=10 proyecciones=10 bins=500: correctas=3502 incorrectas=1498 precision=70.0%	tiempo=5.87	seg. (18.8% del tiempo de Linear Scan)
LSH hashes=10 proyecciones=10 bins=600: correctas=3849 incorrectas=1151 precision=77.0%	tiempo=13.22 seg. (42.4% del tiempo de Linear Scan)
```


### Ejercicio: Comparar con performance obtenida en Dataset B

In [ ]:
experimentoB = Experimento("DATASET_B", "dataset_b_q.npy", "dataset_b_r.npy")
# experimentoB.calcular_linear_scan()

Con LSH se pueden resolver las búsquedas más rápido que el linear scan pagando un costo en la precisión.

**¿Cómo es la relación precision vs eficiencia comparado con los árboles? (ver anexos anteriores)**

Notar que un índice LSH, el nivel de aproximación de la búsqueda (esto es, qué tan rápido va a ir vs qué calidad de respuesta) se decide en la etapa offline (durante la construcción del índice).   
Si se desea mejorar la calidad de respuesta es necesario construir un nuevo índice.